# MP3: Implementing the PLSA Algorithm
## DUE: OCT 23, 2022 at 11:59pm

In this MP, you will implement the PLSA algorithm discussed in lectures [9.7](https://www.coursera.org/learn/cs-410/lecture/HKe8K/9-7-probabilistic-latent-semantic-analysis-plsa-part-1) and [9.8](https://www.coursera.org/learn/cs-410/lecture/GJyGG/9-8-probabilistic-latent-semantic-analysis-plsa-part-2) of the Text Mining Coursera course.
You are not required to implement the PLSA algorithm with a background model (we will run tests assuming the background model has not been implemented). You are provided with two data sets in the `data` folder: `test.txt` and `dblp.txt`. You can assume that each line is a separate document.

The `test.txt` data set contains 1000 lines. Each line is a document. The first 100 documents are labeled with the topic the document belongs to, either 0 (“Seattle”) or 1 (“Chicago”).  Each of the first 100 document is generated by sampling completely from the topic that is labelled (i.e., generated from one topic only). The rest 900 documents are generated from a mixture model of the topic “Seattle” and “Chicago”. Run your code to test if your EM algorithm returns reasonable results.

The `DBLP.txt` data set is made up of research paper abstracts from DBLP. Each line is a document. Make sure to test your code on the simpler data set `test.txt` before running it on `DBLP.txt`.

You are provided with a code skeleton `plsa.py`. Do not change anything in the `def __init__()` function. But feel free to change anything in the `main()` function to test your code.

Some suggested tips:
1.	Using matrices is strongly encouraged (writing nested `for-loops` for calculation is painful)
2.	Python libraries `numpy` and `scipy` are useful in matrix based calculations

#### Writing the PLSA Algorithm:
The original PLSA model does not contain a background model. This MP also is based on the original PLSA model, you do not have to worry about the background model. However, lectures are all about PLSA with a background model, so you should not attempt to map the formulas on lecture slides directly to the code. Instead, you would need to adjust the formulas on slides by assuming that there is zero probability that the background model would be chosen.  That is, you should set λ<sub>B</sub> to zero. If you do this, you will see that the E-step would essentially only compute a distribution over k topics for z=1, ..., k, given each word, i.e., p(z=i|w). The M-step would also be simpler as p(Z=B|w) is also zero for all words (due to the fact that λ<sub>B</sub>=0). If you are still confused, please take a look at Section 3 of Chase Geigle's note on EM [2] and the note below.


The main data structures involved in the implementation of this EM algorithm are three matrices: 
1. T (topics by words): this is the set of parameters characterizing topic content that we denoted by &theta;<sub>i</sub>'s. Each element is the probability of a particular word in a particular topic. 

2. D (documents by topics): this is the set of parameters modeling the coverage of topics in each document, which we denoted by p<sub>ij</sub>'s. Each element is the probability of a particular topic is covered in a particular document. 

3. Z (hidden variables):  For every document, we need one Z which represents the probability that each word in the document has been generated from a particular topic, so for any document, this is a "word-by-topic" matrix, encoding p(Z|w) for a particular document. Z is the matrix that we compute in the E-step (based on matrices T and D, which represent our parameters). Note that we need to compute a different Z for each document, so we need to allocate a matrix Z for every document. If we do so, the M-step is simply to use all these Z matrices together with word counts in each document to re-estimate all the parameters, i.e., updating matrices T and D based on Z. Thus at a high level, this is what's happening in the algorithm: 
    * T and D are initialized. 
    * E-step computes all Z's based on T and D. 
    * M-step uses all Z's to update T and D. 
    * We iterate until the likelihood doesn't change much when we would use T and D as our output. Note that Zs are also very useful (can you imagine some applications of Zs?).

#### Resources:
[1]	[Cheng’s note](http://sifaka.cs.uiuc.edu/czhai/pub/em-note.pdf) on the EM algorithm  
[2]	[Chase Geigle’s note](http://times.cs.uiuc.edu/course/598f16/notes/em-algorithm.pdf) on the EM algorithm, which includes a derivation of the EM algorithm (see section 4), and  
[3]	[Qiaozhu Mei’s note](http://times.cs.uiuc.edu/course/598f16/plsa-note.pdf) on the EM algorithm for PLSA, which includes a different derivation of the EM algorithm.


THIS MP IS CODING AND DEBUGGING HEAVY! PLEASE DON'T LEAVE IT TILL THE LAST MINUTE!

#### Annotations in this programming

Indices: i - doc, j - topic, lw - word

term_doc_matrix c(t | d): c_i,lw

topic_word_prob p(w | z): theta_j,lw

document_topic_prob p(z | d): pi_i,j

topic_prob p(z | d, w): z_i,j,lw

In [1]:
import numpy as np
import math

workspace = '/content/drive/MyDrive/Courses/CS410/MP 3/'

In [2]:
def normalize(input_matrix):
    """
    Normalizes the rows of a 2d input_matrix so they sum to 1
    """

    row_sums = input_matrix.sum(axis=1)
    try:
        assert (np.count_nonzero(row_sums)==np.shape(row_sums)[0]) # no row should sum to zero
    except Exception:
        raise Exception("Error while normalizing. Row(s) sum to zero")
    new_matrix = input_matrix / row_sums[:, np.newaxis]
    return new_matrix

In [3]:
class Corpus(object):

    """
    A collection of documents.
    """

    def __init__(self, documents_path):
        """
        Initialize empty document list.
        """
        self.documents_path = documents_path
        self.documents = []
        self.number_of_documents = 0        
        self.vocabulary = []
        self.vocabulary_size = 0        

        self.term_doc_matrix = None 
        self.document_topic_prob = None  # P(z | d)
        self.topic_word_prob = None  # P(w | z)
        self.topic_prob = None  # P(z | d, w)
        self.likelihoods = []        

    def build_corpus(self):
        """
        Read document, fill in self.documents, a list of list of word
        self.documents = [["the", "day", "is", "nice", "the", ...], [], []...]
        
        Update self.number_of_documents
        """
        # #############################
        # your code here
        f = open(self.documents_path)

        for line in f:
          line = line.strip()
          line = line.split()
          self.documents.append(line)
        
        self.number_of_documents = len(self.documents)
        # #############################
        
        # pass    # REMOVE THIS

    def build_vocabulary(self):
        """
        Construct a list of unique words in the whole corpus. Put it in self.vocabulary
        for example: ["rain", "the", ...]

        Update self.vocabulary_size
        """
        # #############################
        # your code here
        self.vocabulary = list(set([item for sublist in self.documents for item in sublist]))
        self.vocabulary_size = len(self.vocabulary)
        # #############################
        
        # pass    # REMOVE THIS

    def build_term_doc_matrix(self):
        """
        Construct the term-document matrix where each row represents a document, 
        and each column represents a vocabulary term.

        self.term_doc_matrix[i][j] is the count of term j in document i
        """
        # ############################
        # your code here

        # c_il, i: doc, lw: word
        self.term_doc_matrix = np.zeros(shape=(self.number_of_documents,
                                               self.vocabulary_size))
        for (i, doc) in enumerate(self.documents):
          for (lw, word) in enumerate(self.vocabulary):
            self.term_doc_matrix[i,lw] = doc.count(word)
        # ############################
        
        # pass    # REMOVE THIS


    def initialize_randomly(self, number_of_topics):
        """
        Randomly initialize the matrices: document_topic_prob and topic_word_prob
        which hold the probability distributions for P(z | d) and P(w | z): self.document_topic_prob, and self.topic_word_prob

        Don't forget to normalize! 
        HINT: you will find numpy's random matrix useful [https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.random.html]
        """
        # ############################
        # your code here

        # pi_i,j, i: doc, j: topic
        self.document_topic_prob = np.random.random(size=[self.number_of_documents,
                                                          number_of_topics])
        self.document_topic_prob = normalize(self.document_topic_prob)

        # theta_j,lw, j: topic, lw: word
        self.topic_word_prob = np.random.random(size=[number_of_topics,
                                                      self.vocabulary_size])
        self.topic_word_prob = normalize(self.topic_word_prob)
        # ############################

        # pass    # REMOVE THIS
        

    def initialize_uniformly(self, number_of_topics):
        """
        Initializes the matrices: self.document_topic_prob and self.topic_word_prob with a uniform 
        probability distribution. This is used for testing purposes.

        DO NOT CHANGE THIS FUNCTION
        """
        self.document_topic_prob = np.ones((self.number_of_documents, number_of_topics))
        self.document_topic_prob = normalize(self.document_topic_prob)

        self.topic_word_prob = np.ones((number_of_topics, len(self.vocabulary)))
        self.topic_word_prob = normalize(self.topic_word_prob)

    def initialize(self, number_of_topics, random=False):
        """ Call the functions to initialize the matrices document_topic_prob and topic_word_prob
        """
        print("Initializing...")

        if random:
            self.initialize_randomly(number_of_topics)
        else:
            self.initialize_uniformly(number_of_topics)

    def expectation_step(self):
        """ The E-step updates P(z | w, d)
        """
        print("E step:")
        
        # ############################
        # your code here
        for i in range(self.number_of_documents):
          p_z = self.document_topic_prob[i,:][..., None] * self.topic_word_prob  # (j, 1) * (j, lw)
          p_z = normalize(p_z.T).T
          self.topic_prob[i,:,:] = p_z
        
        # topic_prob = []
        # for i in range(self.number_of_documents):
        #   p_z = self.document_topic_prob[i,:] * self.topic_word_prob.T  # (j,) * (j, lw).T
        #   p_z = normalize(p_z)
        #   topic_prob.append(p_z)
        # self.topic_prob = np.array(topic_prob)  # (i, lw, j)
        # ############################

        # pass    # REMOVE THIS
            

    def maximization_step(self, number_of_topics):
        """ The M-step updates P(w | z)
        """
        print("M step:")
        
        # update P(w | z)
        
        # ############################
        # your code here
        for j in range(number_of_topics):
          # theta = np.sum(self.term_doc_matrix * self.topic_prob[:,:,j], axis=0)   # (i, lw) * (i, lw, j=c)
          theta = np.sum(self.term_doc_matrix * self.topic_prob[:,j,:], axis=0)   # (i, lw) * (i, j=c, lw)
          self.topic_word_prob[j,:] = theta
        self.topic_word_prob = normalize(self.topic_word_prob)
        # ############################
        
        # update P(z | d)

        # ############################
        # your code here
        for i in range(self.number_of_documents):
          # pi = self.term_doc_matrix[i,:] @ self.topic_prob[i,:,:]   # (i=c, lw) @ (i=c, lw, j)
          # pi = self.topic_prob[i,:,:] @ self.term_doc_matrix[i,:]  # (i=c, j, lw) @ (i=c, lw)
          pi = np.matmul(self.topic_prob[i,:,:], self.term_doc_matrix[i,:])  # (i=c, j, lw) @ (i=c, lw)
          self.document_topic_prob[i,:] = pi
        self.document_topic_prob = normalize(self.document_topic_prob)
        # ############################
        
        # pass    # REMOVE THIS


    def calculate_likelihood(self, number_of_topics):
        """ Calculate the current log-likelihood of the model using
        the model's updated probability matrices
        
        Append the calculated log-likelihood to self.likelihoods

        """
        # ############################
        # your code here
        # logL = np.sum(self.term_doc_matrix * np.log(self.document_topic_prob @ self.topic_word_prob))  # (i, lw) * ((i, j) @ (j, lw))
        logL = np.sum( self.term_doc_matrix * np.log(np.matmul(self.document_topic_prob, self.topic_word_prob)) )  # (i, lw) * ((i, j) @ (j, lw))
        self.likelihoods.append(logL)
        # ############################
        
        return logL

    def plsa(self, number_of_topics, max_iter, epsilon):

        """
        Model topics.
        """
        print ("EM iteration begins...")
        
        # build term-doc matrix
        self.build_term_doc_matrix()
        
        # Create the counter arrays.
        
        # P(z | d, w)
        # self.topic_prob = np.zeros([self.number_of_documents, number_of_topics, self.vocabulary_size], dtype=np.float)
        self.topic_prob = np.zeros([self.number_of_documents, number_of_topics, self.vocabulary_size])

        # P(z | d) P(w | z)
        self.initialize(number_of_topics, random=True)

        # Run the EM algorithm
        current_likelihood = 0.0

        for iteration in range(max_iter):
            print("Iteration #" + str(iteration + 1) + "...")

            # ############################
            # your code here
            self.expectation_step()
            self.maximization_step(number_of_topics)
            current_likelihood = self.calculate_likelihood(number_of_topics)
            print('current_likelihood: {:.3f}'.format(current_likelihood))

            if iteration == max_iter-1:
              print('==============================')
              print('EM iterations are done!')
              
              # print('vocabulary =', self.vocabulary)
              print('theta_j,lw =')
              # print(self.topic_word_prob.round(3))
              print(np.concatenate([np.array(self.vocabulary)[None, ...], self.topic_word_prob.round(3)], axis=0))
              
              print('pi_i,j =')
              print(self.document_topic_prob[:10].round(3))
            # ############################

            # pass    # REMOVE THIS

In [4]:
# documents_path = 'data/test.txt'
documents_path = workspace + 'data/test.txt'
# documents_path = workspace + 'data/DBLP.txt'
corpus = Corpus(documents_path)  # instantiate corpus
corpus.build_corpus()
corpus.build_vocabulary()
print(corpus.vocabulary)
print("Vocabulary size:" + str(len(corpus.vocabulary)))
print("Number of documents:" + str(len(corpus.documents)))
number_of_topics = 2
max_iterations = 50
epsilon = 0.001
corpus.plsa(number_of_topics, max_iterations, epsilon)

['mount', 'willis', 'chicago', 'seattle', 'rainier', '0', 'tower', '1']
Vocabulary size:8
Number of documents:1000
EM iteration begins...
Initializing...
Iteration #1...
E step:
M step:
current_likelihood: -179013.308
Iteration #2...
E step:
M step:
current_likelihood: -177497.125
Iteration #3...
E step:
M step:
current_likelihood: -175463.495
Iteration #4...
E step:
M step:
current_likelihood: -172528.876
Iteration #5...
E step:
M step:
current_likelihood: -168907.845
Iteration #6...
E step:
M step:
current_likelihood: -165501.648
Iteration #7...
E step:
M step:
current_likelihood: -163107.678
Iteration #8...
E step:
M step:
current_likelihood: -161741.114
Iteration #9...
E step:
M step:
current_likelihood: -161006.971
Iteration #10...
E step:
M step:
current_likelihood: -160605.161
Iteration #11...
E step:
M step:
current_likelihood: -160381.685
Iteration #12...
E step:
M step:
current_likelihood: -160249.948
Iteration #13...
E step:
M step:
current_likelihood: -160162.716
Iteration 